In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load credentials
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

deployment_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

# 1. Define the function (tool)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country, e.g., 'Paris, France'"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# 2. Simulated backend function
def get_weather(location):
    # Replace with real API call if needed
    dummy_weather_data = {
        "Paris, France": "It’s 21°C and sunny in Paris today.",
        "New York, USA": "It’s 28°C and humid in New York today."
    }
    return dummy_weather_data.get(location, f"Weather info not found for {location}.")

# 3. Send initial user message
messages = [{"role": "user", "content": "What's the weather like in Paris, France today?"}]

# 4. Call Azure OpenAI with tool defined
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

message = response.choices[0].message

# 5. Check if the model wants to call a function
if message.tool_calls:
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print(f"\n🛠️ Model wants to call `{function_name}` with:")
    print(arguments)

    # 6. Run the actual function in Python
    if function_name == "get_weather":
        location = arguments.get("location")
        result = get_weather(location)

        # 7. Append tool response to message history
        messages.append(message)  # model's tool_call
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

        # 8. Final call to get the model's full response
        final_response = client.chat.completions.create(
            model=deployment_name,
            messages=messages
        )

        print("\n🤖 Final model response:")
        print(final_response.choices[0].message.content)

else:
    print("\n⚠️ No tool call detected.")



🛠️ Model wants to call `get_weather` with:
{'location': 'Paris, France'}

🤖 Final model response:
It’s currently 21°C and sunny in Paris today.


In [21]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

deployment_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

# Define the tool (function)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and country, e.g. 'Paris, France'"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# Simulated backend function
def get_weather(location):
    # Replace this with an actual API if you want real data
    dummy_weather_data = {
        "Paris, France": "It’s 35°C and sunny in Paris today.",
        "New York, USA": "It’s 28°C and humid in New York today.",
    }
    return dummy_weather_data.get(location, f"Sorry, I don't have weather data for {location}.")

# Step 1: User asks a vague question
messages = [{"role": "user", "content": "What's the weather like?"}]

# Step 2: Let model respond (it should ask for location)
initial_response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

first_model_message = initial_response.choices[0].message
messages.append(first_model_message)

print("\n🤖 Model asks:")
print(first_model_message.content or "[no content, possibly a tool call]")

# Step 3: Simulate user replying with location
user_location_reply = "Paris, France"
messages.append({"role": "user", "content": user_location_reply})

# Step 4: Model now has location, tries to call the function
second_response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

second_model_message = second_response.choices[0].message
messages.append(second_model_message)

# Step 5: Check if tool_call happened
if second_model_message.tool_calls:
    tool_call = second_model_message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print(f"\n🛠️ Model wants to call `{function_name}` with:")
    print(arguments)

    if function_name == "get_weather":
        result = get_weather(arguments["location"])

        # Step 6: Add tool result and ask model for final reply
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": result
        })

        messages.append({
            "role": "system",
            "content": "You are a helpful assistant. Never assume a location. Always ask the user to provide the city and country before calling the weather function."
        })
        
        messages.append({
            "role": "user",
            "content": "What's the weather like in Chennai, India?"
        })

        final_response = client.chat.completions.create(
            model=deployment_name,
            messages=messages
        )

        print("\n🌤️ Final model reply:")
        print(final_response.choices[0].message.content)

else:
    print("\n⚠️ No tool call detected.")



🤖 Model asks:
Please provide me with the location (city and country) you're interested in, so I can fetch the current weather for you!

🛠️ Model wants to call `get_weather` with:
{'location': 'Paris, France'}

🌤️ Final model reply:
Let me check the current weather conditions in Chennai, India for you.
